In [1]:
# used to widen the cells 
from IPython.core.display import display, HTML
display(HTML("<style>.container { margin-left: 2.5% !important; width:95%; }</style>"))


In [2]:
from algosdk.v2client import algod

algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
algod_client = algod.AlgodClient(algod_token, algod_address)


In [3]:
from algosdk import mnemonic, account 

In [4]:
from algosdk import account, mnemonic

def generate_algorand_keypair():
    private_key, address = account.generate_account()
    print("My address: {}".format(address))
    print("My passphrase: {}".format(mnemonic.from_private_key(private_key)))

In [67]:
generate_algorand_keypair()

My address: WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM
My passphrase: angry live wrist arrest nerve stadium round candy scrap area road defy grow panic turkey attack moon shadow they worth shoe forum rural ability average


In [5]:
passphrase = "angry live wrist arrest nerve stadium round candy scrap area road defy grow panic turkey attack moon shadow they worth shoe forum rural ability average"

private_key = mnemonic.to_private_key(passphrase)
my_address = mnemonic.to_public_key(passphrase)
print("My address: {}".format(my_address))

account_info = algod_client.account_info(my_address)
print("Account balance: {} microAlgos".format(account_info.get('amount')))


My address: WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM
Account balance: 99156000 microAlgos


In [6]:
from algosdk.future import transaction
from algosdk.future.transaction import StateSchema, OnComplete

## Deploy

In [7]:
approval_bytes = open('compiled/vote_approval.teal.tok', 'rb').read() 
clear_state_bytes = open('compiled/vote_clear_state.teal.tok', 'rb').read()

In [48]:
params = algod_client.suggested_params()
params.falt_fee = True
params.fee = 1000
sender = my_address

local_schema = StateSchema(num_uints=1, num_byte_slices=0)    # don't fucking touch this. the contract doesn't use it, but it wants it
global_schema = StateSchema(num_uints=20, num_byte_slices=1)
on_complete = OnComplete(0)

app_create_txn = transaction.ApplicationCreateTxn(sender, params, on_complete, approval_bytes, clear_state_bytes, global_schema, local_schema)

In [49]:
app_create_txn_signed = app_create_txn.sign(mnemonic.to_private_key(passphrase))

In [57]:
txid = algod_client.send_transaction(app_create_txn_signed)
print("Successfully sent transaction with txID: {}".format(txid))

AlgodHTTPError: {"message":"TransactionPool.Remember: transaction already in ledger: SEBGAP64NAB3PMPWRM4BZN5LN7CWHPUXKRBP7Y5M6ALGAKR7ECPQ"}


In [56]:
# await confirmation
# wait_for_confirmation(client, txid)   WHAT FUNCTION IS THIS? WHERE DO I FIND IT? 

# since we don't know where to find the above function... you just have to wait a bit 

# display results
transaction_response = algod_client.pending_transaction_info(txid)
app_id = transaction_response['application-index']
print("Created new app-id: ",app_id)


KeyError: 'application-index'

In [31]:
transaction_response

{'application-index': 13061840,
 'confirmed-round': 10118090,
 'global-state-delta': [{'key': 'Q3JlYXRvcg==',
   'value': {'action': 1,
    'bytes': 'tG5RwX0xe3qxgGkxroILazmEnxNyoopGQ+9DkDj7N68='}}],
 'pool-error': '',
 'txn': {'sig': 'NO76lVwv3iUelD5ZLuDOlHScryZmvMWemenR+oPxpPO1/ZSvRwZ6qGeNNgg25ykos8siRFHGetmoq+2L2xgqCQ==',
  'txn': {'apap': 'AiAFAAUEAgEmAgR2b3RlB0NyZWF0b3IxGCISQAAmMRkjEkAAKTEZJBJAACsxGSUSQAAtMRkhBBJAACs2GgAoEkAALAApMQBnIQRDQgA6MQApZBJDQgAxMQApZBJDQgAoIQRDQgAiMQApZBJDQgAZMQApZBJAAAEANhoBNhoBZDYaAhcIZyEEQw==',
   'apgs': {'nbs': 1, 'nui': 2},
   'apls': {'nui': 1},
   'apsu': 'AiABASJD',
   'fee': 384000,
   'fv': 10118088,
   'gen': 'testnet-v1.0',
   'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=',
   'lv': 10119088,
   'snd': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM',
   'type': 'appl'}}}

## Opt in 

In [32]:
optin_tx = transaction.ApplicationOptInTxn(sender, params, app_id)

In [33]:
optin_txid = algod_client.send_transaction(optin_tx.sign(mnemonic.to_private_key(passphrase)))

In [34]:
optin_tx_response = algod_client.pending_transaction_info(optin_txid)
optin_tx_response

{'pool-error': '',
 'txn': {'sig': 'bdy8fm4CPcsTd5IRNelxGsA1VrcaSROpQhJbSTXlnAYCKlw4nUCCiRgXc5+n4KnnwBxVlLmBCwDqJd0oq474BA==',
  'txn': {'apan': 1,
   'apid': 13061840,
   'fee': 217000,
   'fv': 10118088,
   'gen': 'testnet-v1.0',
   'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=',
   'lv': 10119088,
   'snd': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM',
   'type': 'appl'}}}

## Reading global state 

In [35]:
# read app global state
def read_global_state(client, addr, app_id):   
    results = client.account_info(addr)
    apps_created = results['created-apps']
    for app in apps_created :
        if app['id'] == app_id :
            print(f"global_state for app_id {app_id}: ", app['params']['global-state'])

In [36]:
read_global_state(algod_client, my_address, app_id)

global_state for app_id 13061840:  [{'key': 'Q3JlYXRvcg==', 'value': {'bytes': 'tG5RwX0xe3qxgGkxroILazmEnxNyoopGQ+9DkDj7N68=', 'type': 1, 'uint': 0}}]


## Calling with arguments 

In [82]:
# call application with arguments
votes = 17
app_args = ["vote".encode("utf-8"), "candidate".encode("utf-8"), votes.to_bytes(7, "big")]
# call_app(algod_client, user_private_key, app_id, app_args)   again, where tf are these functions? 

# create unsigned transaction
call_tx = transaction.ApplicationNoOpTxn(sender, params, app_id, app_args)
call_txid = algod_client.send_transaction(call_tx.sign(mnemonic.to_private_key(passphrase)))

In [83]:
call_tx_response = algod_client.pending_transaction_info(call_txid)
call_tx_response 

{'confirmed-round': 10117945,
 'global-state-delta': [{'key': 'Y2FuZGlkYXRl',
   'value': {'action': 2, 'uint': 17}}],
 'pool-error': '',
 'txn': {'sig': 'TBTJpvHUpr0nworhstfgJE2OCVYR1JzrNnbj0Lb0C7L8QlZaN7rS9eEnJQgSUwAw73favgVqFOaKNdlC/d0NAw==',
  'txn': {'apaa': ['dm90ZQ==', 'Y2FuZGlkYXRl', 'AAAAAAAAEQ=='],
   'apid': 13061816,
   'fee': 243000,
   'fv': 10117939,
   'gen': 'testnet-v1.0',
   'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=',
   'lv': 10118939,
   'snd': 'WRXFDQL5GF5XVMMANEY25AQLNM4YJHYTOKRIURSD55BZAOH3G6XYHHFCWM',
   'type': 'appl'}}}

## Voting

In [37]:
read_global_state(algod_client, my_address, app_id)

global_state for app_id 13061840:  [{'key': 'Q3JlYXRvcg==', 'value': {'bytes': 'tG5RwX0xe3qxgGkxroILazmEnxNyoopGQ+9DkDj7N68=', 'type': 1, 'uint': 0}}]


In [38]:
def vote(candidate, votes):
    app_args = ["vote".encode("utf-8"), candidate.encode("utf-8"), votes.to_bytes(7, "big")]

    # create unsigned transaction
    call_tx = transaction.ApplicationNoOpTxn(sender, params, app_id, app_args)
    call_txid = algod_client.send_transaction(call_tx.sign(mnemonic.to_private_key(passphrase)))
    

In [39]:
vote("Arthur Dent", 25)

In [41]:
read_global_state(algod_client, my_address, app_id)

global_state for app_id 13061840:  [{'key': 'Q3JlYXRvcg==', 'value': {'bytes': 'tG5RwX0xe3qxgGkxroILazmEnxNyoopGQ+9DkDj7N68=', 'type': 1, 'uint': 0}}, {'key': 'QXJ0aHVyIERlbnQ=', 'value': {'bytes': '', 'type': 2, 'uint': 25}}]


In [42]:
vote("Iron Man", 21)

In [43]:
read_global_state(algod_client, my_address, app_id)

global_state for app_id 13061840:  [{'key': 'Q3JlYXRvcg==', 'value': {'bytes': 'tG5RwX0xe3qxgGkxroILazmEnxNyoopGQ+9DkDj7N68=', 'type': 1, 'uint': 0}}, {'key': 'QXJ0aHVyIERlbnQ=', 'value': {'bytes': '', 'type': 2, 'uint': 25}}, {'key': 'SXJvbiBNYW4=', 'value': {'bytes': '', 'type': 2, 'uint': 21}}]


In [44]:
vote("Someoneelse", 10)

AlgodHTTPError: {"message":"TransactionPool.Remember: transaction OYHBESFW6OOCP7CRY25AG2Y2RDFIB6W3EKCKGAGYIDHIDWCUT47A: store integer count 3 exceeds schema integer count 2"}
